In [3]:
'''
Load data to a database
'''

# from hmmlearn import hmm
import numpy as np
import pandas as pd
from collections import defaultdict
import datetime
# import pykovy-master/ 
# from pykovy.src.pykovy import chain
# import sqlite3
# from sqlalchemy import create_engine

In [4]:
df = pd.read_csv('data_sample/ap.csv')
df = df[['Client Username', 'Association Time','Map Location', 'Session Duration']]

In [5]:
df.head()

,Client Username,Association Time,Map Location,Session Duration
0,164c724c6f2e25d6cd2359216b6b37,Sat Feb 24 00:38:41 PST 2018,Campus Housing > CVB > Floor 11,7 hrs 25 min 57 sec
1,164c724c6f2e25d6cd2359216b6b37,Sat Feb 24 08:04:39 PST 2018,Campus Housing > CVB > Floor 11,8 hrs 10 min 25 sec
2,9913935871e9a863b55e16196eac09,Sat Feb 24 13:16:49 PST 2018,SJSUWIFI > ENG > Floor 4,2 hrs 58 min 15 sec
3,9c508d14b6f397cc7003a91bea4250,Fri Feb 23 16:38:44 PST 2018,SJSUWIFI > ENG > Floor 4,1 hrs 48 min 10 sec
4,9c508d14b6f397cc7003a91bea4250,Fri Feb 23 18:26:55 PST 2018,SJSUWIFI > ENG > Floor 4,10 min 26 sec


In [6]:
def convertMonth(month):
    if month == 'Jan': return 1
    elif month == 'Feb': return 2
    elif month == 'Mar': return 3
    elif month == 'Apr': return 4
    elif month == 'May': return 5
    elif month == 'Jun': return 6
    elif month == 'Jul': return 7
    elif month == 'Aug': return 8
    elif month == 'Sep': return 9
    elif month == 'Oct': return 10
    elif month == 'Nov': return 11
    elif month == 'Dec': return 12
    else: print('error input is not correct')
        
def convert_time_slot(time):
    temp = (time.hour * 60 + time.minute) * 60 + time.second
    return int(temp/(5*60) + 1)

def convert_time(time):
    return time.split(':')

print(convert_time('13:42:56')[0])                              

13


In [8]:
# Get number of users using an AP by date
# need modification for more fine grain time
# input is a panda dataframe
def get_number_of_users_by_date(df):
    df.shape[0]
    temp = df.iloc[0,3].split()
    #temp2 = df.iloc[0,2]
    #temp2
    result = {}
    client_list = {}
    for i in range(df.shape[0]):
        association_time = df.iloc[i,1].split()
        temp1 = df.iloc[i,3].split()
        AP_name = df.iloc[i,2]
        client_username = df.iloc[i,0]
    
        if len(temp1) == 2: sess_duration = int(temp1[0])
        elif len(temp1) == 4: sess_duration = int(temp1[0]) * 60 + int(temp1[2])
        elif len(temp1) == 6: sess_duration = (int(temp1[0]) * 60 + int(temp1[2])) * 60 + int(temp1[4])
        else: print('error')

        d_time = datetime.datetime(year=int(association_time[5]), 
                                   month=convertMonth(association_time[1]), 
                                   day=int(association_time[2]))
        
        if AP_name not in result:
            # new AP, just add it to the list
            day_dict = {}
            day_dict[str(d_time)] = 1
            result[AP_name] = day_dict
            client_list[client_username] = []
            client_list[client_username].append(str(d_time))
        else:
            if result[AP_name].get(str(d_time)):
                # time entry already in the list
                # need to check if the user is already counted
                # if not, then add this in the list
                if client_username not in client_list:
                    result[AP_name][str(d_time)] += 1
                    client_list[client_username] = []
                    client_list[client_username].append(str(d_time))
                    #client_list[client_username] = 1
                else:
                    if str(d_time) not in client_list[client_username]:
                        result[AP_name][str(d_time)] += 1
                        client_list[client_username].append(str(d_time))
            else:
                # new time entry, just add it to the list
                result[AP_name][str(d_time)] = 1
                client_list[client_username] = []
                client_list[client_username].append(str(d_time))
                #client_list[client_username] = 1
    
    print(client_list)
    return result   
    #data_dict2[df.iloc[i,0]].append([d_time, df.iloc[i,2], sess_duration])

output = get_number_of_users_by_date(df)
output['Campus Housing > CV2 > 4th floor']

{'9d1a4172a3ab3812d1567347a5397c': ['2018-02-24 00:00:00'], 'c11876629f43423f182fb966246496': ['2018-02-24 00:00:00'], '656d29b0b45c23e74ff14195a3297d': ['2018-02-23 00:00:00'], '75cb957510e0ce6f67afca9583e9cc': ['2018-02-23 00:00:00', '2018-02-24 00:00:00'], '43455986f5281f7acecf02dc377f7b': ['2018-02-23 00:00:00', '2018-02-24 00:00:00'], 'b153e827cebf86db4c7ea1dff3a0f8': ['2018-02-24 00:00:00'], 'eabe0a235883b232f70ae7f25f12d3': ['2018-02-23 00:00:00'], '164c724c6f2e25d6cd2359216b6b37': ['2018-02-24 00:00:00'], 'e136f6a832f9efb58f11fbbd17ffaa': ['2018-02-23 00:00:00'], 'd6f66f6ef442dbab938fc4a93f2e5d': ['2018-02-24 00:00:00'], '24d3bfb47233cb06c74c7196dbebd6': ['2018-02-24 00:00:00'], 'ddf266536ce2f6d676e3171e1e3121': ['2018-02-24 00:00:00'], '35d2ecdd1707938cb08b1661ff1f4d': ['2018-02-23 00:00:00', '2018-02-24 00:00:00'], 'a84a577666e68a43a6ab5ed44aebf4': ['2018-02-24 00:00:00'], 'e656b68788bdfcb36485d01daff436': ['2018-02-24 00:00:00'], '49c4625afdcd3cf0c7f945ac41ad5f': ['2018-02-2

{'2018-02-23 00:00:00': 3, '2018-02-24 00:00:00': 3}

In [37]:
# Get number of users using an AP by date
# need modification for more fine grain time
# input is a panda dataframe
# TODO: need to check for user_name
def get_number_of_users_by_date(df):
    df.shape[0]
    temp = df.iloc[0,3].split()
    #temp2 = df.iloc[0,2]
    #temp2
    result = {}
    client_list = {}
    for i in range(df.shape[0]):
        association_time = df.iloc[i,1].split()
        temp1 = df.iloc[i,3].split()
        AP_name = df.iloc[i,2]
        client_username = df.iloc[i,1]
    
        if len(temp1) == 2: sess_duration = int(temp1[0])
        elif len(temp1) == 4: sess_duration = int(temp1[0]) * 60 + int(temp1[2])
        elif len(temp1) == 6: sess_duration = (int(temp1[0]) * 60 + int(temp1[2])) * 60 + int(temp1[4])
        else: print('error')

        d_time = datetime.datetime(year=int(association_time[5]), 
                                   month=convertMonth(association_time[1]), 
                                   day=int(association_time[2]),
                                   hour=int(convert_time(association_time[3])[0]))
        
        if AP_name not in result:
            # new AP, just add it to the list
            day_dict = {}
            day_dict[str(d_time)] = 1
            result[AP_name] = day_dict
            client_list[client_username] = 1
        else:  
            if result[AP_name].get(str(d_time)):
                # time entry already in the list
                # need to check if the user is already counted
                # if not, then add this in the list
                if client_username not in client_list:
                    result[AP_name][str(d_time)] += 1
                    client_list[client_username] = 1
            else:
                # new time entry, just add it to the list
                result[AP_name][str(d_time)] = 1
                client_list[client_username] = 1
            
    return result   
    #data_dict2[df.iloc[i,0]].append([d_time, df.iloc[i,2], sess_duration])

output = get_number_of_users_by_date(df)
output['Campus Housing > CV2 > 4th floor']

{'2018-02-23 17:00:00': 1,
 '2018-02-23 18:00:00': 1,
 '2018-02-23 19:00:00': 1,
 '2018-02-23 20:00:00': 1,
 '2018-02-23 21:00:00': 1,
 '2018-02-23 22:00:00': 3,
 '2018-02-23 23:00:00': 1,
 '2018-02-24 00:00:00': 1,
 '2018-02-24 11:00:00': 1,
 '2018-02-24 12:00:00': 2,
 '2018-02-24 13:00:00': 1,
 '2018-02-24 14:00:00': 1,
 '2018-02-24 15:00:00': 1,
 '2018-02-23 16:00:00': 1,
 '2018-02-24 01:00:00': 1,
 '2018-02-24 02:00:00': 1,
 '2018-02-24 03:00:00': 1}